# GIF Generation with Wave.js

This notebook allows to record rotating GIFs of materials using the Wave.js viewer.

## Usage
1. Set the URL of the Wave.js viewer (either local or deployed).
2. Set the folder containing the JSON files of the materials.
3. Run the notebook to generate GIFs for each material in the specified folder.


## 1. Settings of Notebook

In [ ]:
URL = "https://exabyte-io.github.io/wave.js/" # or
# URL = "http://localhost:3002/"
JSON_FOLDER = "structures"

## 2. Loading of materials

In [ ]:
import os, json


def load_json_files_from_folder(folder_path):
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    json_data = []
    file_names = []

    for json_file in json_files:
        file_path = os.path.join(folder_path, json_file)
        with open(file_path, 'r') as f:
            data = json.load(f)
            json_data.append(data)
            file_names.append(os.path.splitext(json_file)[0])

    return json_data, file_names


json_data, file_names = load_json_files_from_folder(JSON_FOLDER)

materials_settings = []
for data in json_data:
    materials_settings.append({
        "material_json": data,
        "name": file_names[json_data.index(data)]  # name of the file
        # "name": data["name"] # name of the material
    })

## 3. Actions Definitions

In [ ]:
from IPython.display import display, IFrame, Javascript


def send_message_to_iframe(message):
    js_code = f"""
    (function() {{
        const iframe = document.querySelector('iframe');
        if (iframe && iframe.contentWindow) {{
            try {{
                iframe.contentWindow.postMessage({json.dumps(message)}, '*');
                console.log('Message sent to iframe');
            }} catch (error) {{
                alert('Error sending message to iframe. See console for more information.', error);
            }}
        }} else {{
            console.error('Iframe not found or not ready');
        }}
    }})();
    """
    display(Javascript(js_code))


def set_material_in_iframe(material_json):
    """
    Uses send_message_to_iframe to send a material configuration
    under the "material" key to the iframe at `url`.
    """
    message = {
        "action": "handleSetMaterial",
        "parameters": [material_json]
    }
    send_message_to_iframe(message)


def record_gif(filename, rotation_speed=60, frame_duration=0.05):
    """
    Uses send_message_to_iframe to send a message to the iframe at `url`
    to start recording a GIF of the visualization with the specified parameters.
    """
    message = {
        "action": "handleStartGifRecording",
        "parameters": [filename, rotation_speed, frame_duration]
    }
    send_message_to_iframe(message)


def set_camera(pos=(15, 15, 15), target=(0, 0, 0)):
    func_str = f"camera.position.set({pos[0]},{pos[1]},{pos[2]})"
    message = {
        "action": "doWaveFunc",
        "parameters": [func_str]
    }
    send_message_to_iframe(message)


def set_camera_to_fit_cell():
    message = {
        "action": "handleSetCameraToFitCell",
        "parameters": [None]
    }
    send_message_to_iframe(message)


def toggle_bonds():
    message = {
        "action": "handleToggleBonds",
        "parameters": [None]
    }
    send_message_to_iframe(message)


def handle_reset_viewer():
    message = {
        "action": "handleResetViewer",
        "parameters": [None]
    }
    send_message_to_iframe(message)


def test_do_func():
    func_str = "camera.position.set(0,16,0)"
    message = {
        "action": "doWaveFunc",
        "parameters": [func_str]
    }
    send_message_to_iframe(message)


## 4. Gif Generation

### 4.1. Load the iframe with the Wave.js viewer

In [ ]:
# Example usage with a material configuration:
iframe = IFrame(src=URL, width="600", height="600")
display(iframe)


### 4.2. Generate GIFs for each material

In [ ]:
import time

for material_settings in materials_settings:
    GIF_NAME = material_settings["name"] + ".gif" or material_settings["material_json"]["name"] + ".gif"
    handle_reset_viewer()
    time.sleep(1)
    set_material_in_iframe(material_settings["material_json"])
    time.sleep(1)
    toggle_bonds()
    time.sleep(1)
    set_camera_to_fit_cell()
    time.sleep(1)
    record_gif(GIF_NAME)
    # We should wait until the gif is generated and saved before moving to the next material
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
    gif_path = os.path.join(parent_dir, GIF_NAME)
    print(f"Waiting for gif to be generated in {gif_path}")

    # Wait until the gif file is created and downloaded
    while not os.path.exists(gif_path):
        time.sleep(1)
    else:
        print(f"Generated gif for {GIF_NAME}")
    # time.sleep(30)
